# Model Comparisons


In [12]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/my-stuff/projects/neural network from scratch/src')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/my-stuff/projects/neural network from scratch/src'

In [14]:
import datetime
from functools import wraps
import time

class ExecutionTimer:
    def __init__(self) -> None:
        self.execution_times = {}

    def __call__(self, fn):
        """
        Allows the class to be used as a decorator.
        """
        @wraps(fn)
        def wrapper(*args, **kwargs):
            start = datetime.datetime.now()
            result = fn(*args, **kwargs)
            end = datetime.datetime.now()
            execution_time = (end - start).total_seconds()
            self.execution_times[fn.__name__] = execution_time
            print(f"{fn.__name__} execution time: {execution_time:.2f} seconds")
            return result
        return wrapper

timer= ExecutionTimer()


## Our library

In [13]:
import numpy as np
from matplotlib import pyplot as plt
from termcolor import colored

from module import Module
from linear import Linear
from optimizer import SGD
from loss import CrossEntropyLoss, MSE
from activation import ReLU, Softmax
from dataset import MNIST
from dataloader import DataLoader
from transforms import Compose, ToTensor, Normalize, Standardize
from tensor import Tensor

In [3]:
# -- using our implemented dataset module
transformation=Compose([ToTensor(), Standardize()])
train_data = MNIST(root='data/', train=True, download=True,transform=transformation)
test_data = MNIST(root='data/', train=False, download=True,transform=transformation)

# -- using our implemented dataloader module
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=32, shuffle=True)

 >>> data/MNIST/raw/t10k-labels-idx1-ubyte.gz already exists <<<
 >>> data/MNIST/raw/t10k-labels-idx1-ubyte already exists <<<
 >>> data/MNIST/raw/train-labels-idx1-ubyte.gz already exists <<<
 >>> data/MNIST/raw/train-labels-idx1-ubyte already exists <<<
 >>> data/MNIST/raw/t10k-images-idx3-ubyte.gz already exists <<<
 >>> data/MNIST/raw/t10k-images-idx3-ubyte already exists <<<
 >>> data/MNIST/raw/train-images-idx3-ubyte.gz already exists <<<
 >>> data/MNIST/raw/train-images-idx3-ubyte already exists <<<
>>> applying ToTensor()...
 :O already a tensor
>>> applying Standardize(inplace=True)...
>>> [ToTensor(), Standardize(inplace=True)] applied successfully <<<
 >>> data/MNIST/raw/t10k-labels-idx1-ubyte.gz already exists <<<
 >>> data/MNIST/raw/t10k-labels-idx1-ubyte already exists <<<
 >>> data/MNIST/raw/train-labels-idx1-ubyte.gz already exists <<<
 >>> data/MNIST/raw/train-labels-idx1-ubyte already exists <<<
 >>> data/MNIST/raw/t10k-images-idx3-ubyte.gz already exists <<<
 >>> dat

In [7]:
# timer()
# def train_network_our_library():
#     # -- model definition
#     class Model(Module):
#         def __init__(self):
#             super().__init__()
#             self.linear1 = Linear(28*28,20)
#             self.relu=ReLU()
#             self.linear2 = Linear(20, 10)
#             self.softmax=Softmax()

#         def forward(self, x):
#             x = self.linear1(x)
#             x = self.relu(x)
#             x = self.linear2(x)
#             return self.softmax(x)

#     model = Model()
#     optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
#     loss_fn = CrossEntropyLoss()

#     accuracies = []

#     # -- running the experiment 10 times
#     for run in range(10):
#         print(f"Run {run + 1} / 10:")

#         # -- training loop
#         for epoch in range(1):  # You can adjust the number of epochs as needed
#             for batch_no, (x, y) in enumerate(train_loader):
#                 # -> flatten the batch (32, 1, 28, 28) to (784, 32)
#                 x = x.flatten_batch()  # (784, 32)

#                 optimizer.zero_grad()
#                 y_hat = model(x)
#                 loss = loss_fn(y, y_hat)
#                 loss.backward()
#                 optimizer.step()
#         # -- testing
#         correct = 0
#         total = 0

#         for batch_no, (x, y) in enumerate(test_loader):
#             x = x.flatten_batch()
#             y_hat = model(x)
#             predictions = np.argmax(y_hat, axis=0)
#             correct += np.sum(predictions == y)
#             total += y.data.size

#         accuracy = correct / total * 100
#         accuracies.append(accuracy)

#         print(f'Accuracy for run {run + 1}: {accuracy:.2f}%')
#         print('------------------')
#         return

In [19]:
@timer
def train_network_our_library():

    # -- model definition
    class Model(Module):
        def __init__(self):
            super().__init__()
            self.linear1 = Linear(28*28,20)
            self.relu=ReLU()
            self.linear2 = Linear(20, 10)
            self.softmax=Softmax()

        def forward(self, x):
            x = self.linear1(x)
            x = self.relu(x)
            x = self.linear2(x)
            return self.softmax(x)

    model = Model()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    loss_fn = CrossEntropyLoss()

    train_accuracies = []; epoch_train_accuracies=[]
    test_accuracies = [];

    # -- running the experiment 10 times
    for run in range(10):
        print(colored(f"{'='*10} Run {run + 1} / 10 {'='*10}", 'cyan', attrs=['bold']))

        # -- training loop
        correct_train = 0
        total_train = 0
        for epoch in range(10):
            for batch_no, (x, y) in enumerate(train_loader):
                # -> flatten the batch (32, 1, 28, 28) to (784, 32)
                x = x.flatten_batch()  # (784, 32)

                optimizer.zero_grad()
                y_hat = model(x)
                loss = loss_fn(y, y_hat)
                loss.backward()
                optimizer.step()


                predictions = np.argmax(y_hat, axis=0)
                correct_train += np.sum(predictions == y)
                total_train += y.data.size

                epoch_train_accuracy = correct_train / total_train * 100
                epoch_train_accuracies.append(epoch_train_accuracy)

        train_accuracy = correct_train / total_train * 100
        train_accuracies.append(train_accuracy)
        print(colored(f"🎯 Training Accuracy: {train_accuracy:.2f}%", 'green'))

        # -- testing loop
        correct_test = 0
        total_test = 0
        for batch_no, (x, y) in enumerate(test_loader):
            x = x.flatten_batch()
            y_hat = model(x)
            predictions = np.argmax(y_hat, axis=0)
            correct_test += np.sum(predictions == y)
            total_test += y.data.size

        test_accuracy = correct_test / total_test * 100
        test_accuracies.append(test_accuracy)
        print(colored(f"🔍 Testing Accuracy: {test_accuracy:.2f}%", 'yellow'))
        print(colored('-'*30, 'magenta'))

    print(colored(f"\n{'='*15} Summary of Results {'='*15}", 'blue', attrs=['bold']))
    for i in range(10):
        print(colored(f"Run {i + 1} - 🏋️ Training: {train_accuracies[i]:.2f}% | 🧪 Testing: {test_accuracies[i]:.2f}%", 'white'))
    print(colored(f"{'='*50}", 'blue', attrs=['bold']))
    return train_accuracies, epoch_train_accuracies, test_accuracies


In [20]:
train_acc, epoch_train_acc, test_acc = train_network_our_library()



========== Run 1 / 10 ==========
🎯 Training Accuracy: 93.11%
🔍 Testing Accuracy: 94.58%
------------------------------
========== Run 2 / 10 ==========
🎯 Training Accuracy: 96.04%
🔍 Testing Accuracy: 94.59%
------------------------------
========== Run 3 / 10 ==========
🎯 Training Accuracy: 96.75%
🔍 Testing Accuracy: 94.86%
------------------------------
========== Run 4 / 10 ==========
🎯 Training Accuracy: 97.15%
🔍 Testing Accuracy: 95.24%
------------------------------
========== Run 5 / 10 ==========
🎯 Training Accuracy: 97.39%
🔍 Testing Accuracy: 95.01%
------------------------------
========== Run 6 / 10 ==========
🎯 Training Accuracy: 97.61%
🔍 Testing Accuracy: 94.89%
------------------------------
========== Run 7 / 10 ==========
🎯 Training Accuracy: 97.79%
🔍 Testing Accuracy: 95.01%
------------------------------
========== Run 8 / 10 ==========
🎯 Training Accuracy: 97.94%
🔍 Testing Accuracy: 94.52%
------------------------------
========== Run 9 / 10 ==========
🎯 Training Accu

In [23]:
average_train_accuracy_epochs = sum(epoch_train_acc) / len(epoch_train_acc)
print(f'Average Train Accuracy over 10 runs 10 epochs each: {average_train_accuracy_epochs:.2f}%')

average_test_accuracy = sum(test_acc) / len(test_acc)
print(f'Average Test Accuracy over 10 runs: {average_test_accuracy:.2f}%')

Average Train Accuracy over 10 runs 10 epochs each: 96.55%
Average Test Accuracy over 10 runs: 94.86%


## PyTorch

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testset = datasets.MNIST('data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)

In [36]:
# @timer
# def train_network_pytorch():

#     model = nn.Sequential(nn.Linear(28*28, 20), nn.ReLU(), nn.Linear(20, 10), nn.Softmax(dim=1))
#     optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#     loss_fn = nn.CrossEntropyLoss()

#     train_accuracies = []  # To store training accuracies
#     test_accuracies = []   # To store testing accuracies
#     epochs_list = []       # To store number of epochs used in each run

#     # Run the experiment 10 times
#     for run in range(10):
#         print(f"Run {run + 1} / 10:")

#         # Initialize variables for this run
#         epoch_accuracies_train = []  # List to store per-epoch training accuracy

#         # -- training loop
#         model.train()  # Set the model to training mode
#         running_loss = 0.0
#         correct_train = 0
#         total_train = 0

#         for epoch in range(10):  # You can adjust the number of epochs as needed
#             # epoch_accuracies_train = []

#             for batch_no, (x, y) in enumerate(trainloader):
#                 x = x.view(x.shape[0], -1)
#                 optimizer.zero_grad()
#                 y_hat = model(x)
#                 loss = loss_fn(y_hat, y)
#                 loss.backward()
#                 optimizer.step()

#                 running_loss += loss.item()  # Keep track of the loss for training
#                 predictions = torch.argmax(y_hat, dim=1)
#                 correct_train += torch.sum(predictions == y)
#                 total_train += y.size(0)

#             # Calculate and store the training accuracy for this epoch
#             train_accuracy = correct_train / total_train * 100
#             epoch_accuracies_train.append(train_accuracy)

#         # Store the final training accuracy for this run
#         train_accuracies.append(epoch_accuracies_train[-1])
#         epochs_list.append(1)  # You can adjust the number of epochs as needed
#         print(f'Epoch {epoch + 1}, Training Loss: {running_loss / len(trainloader):.4f}')
#         print(f'Training Accuracy for run {run + 1}: {train_accuracies[-1]:.2f}%')

#         # -- testing loop
#         model.eval()  # Set the model to evaluation mode
#         correct_test = 0
#         total_test = 0

#         with torch.no_grad():  # Disable gradient calculation during testing
#             for batch_no, (x, y) in enumerate(testloader):
#                 x = x.view(x.shape[0], -1)
#                 y_hat = model(x)
#                 predictions = torch.argmax(y_hat, dim=1)
#                 correct_test += torch.sum(predictions == y)
#                 total_test += y.size(0)

#         # Calculate and store the testing accuracy
#         test_accuracy = correct_test / total_test * 100
#         test_accuracies.append(test_accuracy)

#         print(f'Pytorch Accuracy for run {run + 1} (Testing): {test_accuracy:.2f}%')
#         print('------------------')
#         return train_accuracies, epochs_list, test_accuracies

In [ ]:
torch_train_acc, torch_epoch_train_acc, torch_test_acc = train_network_pytorch()


Run 1 / 10:


In [ ]:
print(colored(f"\n{'='*15} Summary of Results {'='*15}", 'blue', attrs=['bold']))
for i in range(10):
    print(colored(f"Run {i + 1} - 🏋️ Training: {torch_train_acc[i]:.2f}% | 🧪 Testing: {torch_test_acc[i]:.2f}%", 'white'))
    print(colored(f"{'='*50}", 'blue', attrs=['bold']))


In [ ]:
# print(f"Average Training Accuracy across all runs: {sum(train_accuracies) / len(train_accuracies):.2f}%")
# print(f"Average Testing Accuracy across all runs: {sum(test_accuracies) / len(test_accuracies):.2f}%")
# print(f"Epochs used in each run: {epochs_list}")

In [27]:
# torch_average_train_accuracy_epochs = sum(torch_epoch_train_acc) / len(torch_epoch_train_acc)
# print(f'torch Average Train Accuracy over 10 runs 10 epochs each: {torch_average_train_accuracy_epochs:.2f}%')

torch_average_train_accuracy = sum(torch_train_acc) / len(torch_train_acc)
print(f'torch Average Train Accuracy over 10 runs: {torch_average_train_accuracy:.2f}%')

torch_average_test_accuracy = sum(torch_test_acc) / len(torch_test_acc)
print(f'torch Average Test Accuracy over 10 runs: {torch_average_test_accuracy:.2f}%')

torch Average Train Accuracy over 10 runs 10 epochs each: 96.79%
torch Average Test Accuracy over 10 runs: 95.19%


In [ ]:
import matplotlib.pyplot as plt

# Setup the figure for plotting
plt.figure(figsize=(14, 6))

# Plot 1: torch_test_acc vs test_acc
plt.subplot(1, 2, 1)  # 1 row, 2 columns, plot 1
plt.plot(torch_test_acc, label='Torch Test Accuracy', color='green', linestyle='-', marker='o', markersize=5)
plt.plot(test_acc, label='Other Test Accuracy', color='orange', linestyle='--', marker='x', markersize=5)
plt.title('Test Accuracy Comparison', fontsize=14)
plt.xlabel('Runs (10 runs)', fontsize=12)
plt.ylabel('Accuracy (%)', fontsize=12)
plt.legend()
plt.grid(True)

# Plot 2: torch_train_acc vs train_acc
plt.subplot(1, 2, 2)  # 1 row, 2 columns, plot 2
plt.plot(torch_train_acc, label='Torch Train Accuracy', color='red', linestyle='-', marker='o', markersize=5)
plt.plot(train_acc, label='Other Train Accuracy', color='blue', linestyle='--', marker='x', markersize=5)
plt.title('Train Accuracy Comparison', fontsize=14)
plt.xlabel('Runs (10 runs)', fontsize=12)
plt.ylabel('Accuracy (%)', fontsize=12)
plt.legend()
plt.grid(True)

# Adjust layout for better spacing
plt.tight_layout()

# Show the plot
plt.show()

## Comparisons

In [28]:
timer.execution_times

{'train_network_our_library': 312.861837}